# Overview of Electricity Generation in the USA based on EIA data

The following is an exercise in using data from the Electricity Information Agency (EIA),
to famialarize myself with both the broad state of the US electrical grid, as well as some elementary data science.

I have converted the bulk data file from (url_here) to a local SQL database.  I used Psycopg2 for direct SQL interaction with that PostGRESQL database.  
I can then efficiently select data from that database, based on geographic region and type of generation.  (While the data should nominally fit into memory in one Pandas dataframe, I found that my computer ran out of memory before it could be loaded.)  

The resulting queries from SQL can then be converted into a Pandas dataframe.  Since the data is of the form
"[[date1,number1],[date2,number]...], as one long string, this is converted into a list, then a Numpy array.
The first column is used to generate a Pandas PeriodIndex (which is then further converted to a DateTimeIndex for plotting reasons).  The second column is then indexed.  The resulting series is output as the elements of a list of series for the desired state, and generation type.  
These results can then be plotted using basic Matplotlib.

In [3]:
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2
import datetime 
import matplotlib.pyplot as pl
from psycopg2 import sql

from sql_lib import create_conn_and_cur

In [5]:
#Set up connections to SQL database, which was generated from the bulk JSON file.
database_name='US_ELEC'
fname='ELEC'
table_name=fname
engine=sqlalchemy.create_engine("postgresql+psycopg2://localhost/"+database_name)

#make connection to database (which is needed by Pandas), 
# and the cursor which actually executes SQL commands.
conn=psycopg2.connect(dbname=database_name,host='localhost')
conn.set_session(autocommit=True)
cur = conn.cursor()


The following defines some useful functions for grabbing SQL queries, and loading the desired columns into a Pandas DataFrame.

In [28]:
#make safe SQL queries, with deired list of columns in "out_columns".
#Assume we are searching through name for entries with desired type of series, for particular states,
#as well as generation type.
def safe_sql_query(table_name,
                   out_columns,
                   series_type='Net generation', 
                   state='Oregon',
                   sector='all sectors',
                   gen_type='hydro',
                   freq='M'):
    #make up categories to match the name by. 
    query_match_list=list()
    l1 = sql.Literal(series_type+' :%')
    l2 = sql.Literal('%: '+state+' :%')
    l3 = sql.Literal('%'+gen_type+'%')
    l4 = sql.Literal('%'+sector+'%')
    #join together these matches with ANDs to match them all
    like_query=sql.SQL(' AND name LIKE ').join([l1,l2,l3,l4])

    #Total SQL query to select desired columns with features 
    q1 = sql.SQL("SELECT {} FROM {} WHERE (name LIKE {} AND f LIKE {}) ").format(
                sql.SQL(' ,').join(map(sql.Identifier,out_columns)),
                sql.Identifier(table_name),
                like_query,
                sql.Literal(freq))

    return(q1)

#Get a dataframe from SQL database for given psycopg2 cursor,
#with desired output columns.     
#Must select data based on series type, state, and type of generation.
def get_dataframe(cur,
                  out_columns,
                  table="ELEC",
                  series_type='Net Generation',
                  state='Oregon',
                  sector='all sectors',
                  gen_type='solar',
                  freq='M'):
    q = safe_sql_query(table,out_columns,series_type=series_type,state=state,sector=sector,gen_type=gen_type,freq=freq)
    cur.execute(q);
    df0=cur.fetchall();
    df = pd.DataFrame(df0,columns=out_columns);
    return df


In [22]:
#Initial readin of SQL dataframes returns 'data' as a string of a list of lists.  
#This function goes row by row, converting that 'data' column
#into a new series, with datetimeindex in 'data2'

# def convert_df(df):
#     Nrows=len(df)
# # df['data2']=pd.Series()
#     data_array=[];
#     for i in range(0,Nrows):
#         #check there's actually data there.
#         print('Making',i,'dataset')
#         #use next line since the read in dataframe has returned a string.
#         init_series=np.asarray(eval(df.loc[i,'data']))
#         dat2=init_series[:,1];
#         timeindex=make_timeindex(df.loc[i])
#         if (len(dat2) != len(timeindex)):
#             print('Unequal lengths')
#             print(len(dat2),len(timeindex))
#         else:
#             s=pd.Series(dat2,index=timeindex)
#             data_array.append(s)

#         return data_array

    
# Make a Period Index - really, really easy.
#But plotting is limited with "Periods".  It seems only
#"DateTimeIndices" allow easy combinations.  
def convert_df(df):
    Nrows=len(df)
# df['data2']=pd.Series()
    data_array=[];
    for i in range(0,Nrows):
        #check there's actually data there.
        print('Making',i,'dataset')
        #use next line since the read in dataframe has returned a string.
        init_series=np.asarray(eval(df.loc[i,'data']))
        dat2=init_series[:,1];
        f = df.loc[i,'f']
        periodindex=pd.PeriodIndex(init_series[:,0],freq=f)
        s=pd.Series(dat2,index=periodindex)
        data_array.append(s.to_timestamp())

    return data_array

In [29]:
#Try to select data from fields with names such as net generation and desired state.
#Possibly also by electricity source.  Nuclear/Solar/Wind/Gas/Coal etc.  

#Useful fields: Net Generation : state : type
#Also interesting: Average retail price of electricity.  
#Net generation
#Retail sales of electricity
#Revenue

#Can Identify useful tags by splitting at colons":"
out_col=('name','data','start','end','f')
df=get_dataframe(cur,out_col,series_type='Net generation',state='United States',gen_type='coal');
len(df)
d0=convert_df(df)
df['data2']=d0

Making 0 dataset


In [31]:
df['data2']

0    2016-10-01     99348.27191
2016-09-01    114279.66583
2016-08-01    135808.99792
2016-07-01    1...
Name: data2, dtype: object

In [32]:
for i in range(0,len(d0)):
    pl.plot(d0[i],label=df.loc[i,'name'])
pl.legend(loc='upper left',bbox_to_anchor=(1,1))
pl.show()

In [16]:
df['data2']=d0



In [20]:
df.loc[3,'data2']

2016-10-01     867.23907
2016-09-01    1062.51575
2016-08-01    1425.55774
2016-07-01    1428.91113
2016-06-01    1135.71603
2016-05-01     982.12739
2016-04-01     819.61795
2016-03-01    1186.26312
2016-02-01    1314.56729
2016-01-01    1460.75627
2015-12-01    1382.72935
2015-11-01    1372.23116
2015-10-01      1438.387
2015-09-01    1496.14432
2015-08-01    1657.91515
2015-07-01    1550.12742
2015-06-01    1505.57156
2015-05-01    1361.32943
2015-04-01    1070.60848
2015-03-01    1304.99344
2015-02-01    1573.32056
2015-01-01     1365.9259
2014-12-01    1429.56847
2014-11-01    1552.04534
2014-10-01    1522.16556
2014-09-01    1614.24824
2014-08-01    1823.91512
2014-07-01     1840.8166
2014-06-01    1768.58692
2014-05-01    1515.59692
                 ...    
2003-06-01      3015.975
2003-05-01      2736.742
2003-04-01      2631.397
2003-03-01      2927.808
2003-02-01      3040.436
2003-01-01      3290.646
2002-12-01      2719.324
2002-11-01      2268.341
2002-10-01      2283.852


Finally (months after I first wanted to do this), we can start looking at some data.  
I would like to review the sources of electricity to get a sense of seasonal variations (in terms of availability and use).  For example, I'd expect solar generation to be largest in summer, lowest in winter.  
I would assume this varies geographically.  I'll assume the seasonality can be averaged over by using the annual data. 

Residential Use will be largest in winter, barring the occasional fluctuation in summer.  

The purpose of this survey is to identify the scale of renewables in the market.  
How large a share do renewables (solar, hydro, wind) take up?  How does this vary regionally?  Over time? 
Who are the primary users of electricity?  
I'll assume the base load is provided by coal, gas and nuclear plants.  


Ultimately however, I intend to try my hand at some machine learning projects relevant to electricity.  
The first project involves demand forecasting.  The other place where data science (and nifty math) may have a role in this, is on the market.  While I am comfortable with stochastic calculus etc, I would need to learn about options pricing
and the relevant techniques for this sector of the energy market.

This is meant to be a brief background piece for that.  Since we are forecasting a time series,
I aim to use TensorFlow with some form of neural network (perhaps recurrent) to try forecasting.
Ideally, I would cross-reference the electricity data sets against the published weather forecasts for a given day.  
The model should be able to predict demand given the location, time of year, and weather forecast.  (This is probably the 
simplest thing available)

From a smart-grid perspective, how can randomly fluctuating sources, such as solar and wind, be included 
in supplying power to the grid?  